<a href="https://colab.research.google.com/github/Zabiullahkhan/Data_Science/blob/main/TEST_DISTILLBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
!pip install accelerate -U

!pip  install transformers
import transformers
print(transformers.__version__)

4.33.2


In [2]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import DistilBertTokenizer
# from transformers import Pipelline, TextClassificationPipeline
from transformers import DistilBertTokenizer, DistilBertModel
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
from tqdm import tqdm

In [3]:
# !wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/train.csv
# !wget https://raw.githubusercontent.com/PolyAI-LDN/task-specific-datasets/master/banking_data/test.csv
# !wget https://github.com/PolyAI-LDN/task-specific-datasets/blob/master/banking_data/categories.json


train_data = pd.read_csv('../content/train.csv')
test_data = pd.read_csv('../content/test.csv')
df = pd.concat([train_data, test_data], axis=0)
df = df.reset_index(drop=True)
df['label'] = df['category'].astype('category').cat.codes
id2label = {label:category for category, label in sorted(df[['category', 'label']].drop_duplicates().values, key = lambda x:x[1])}
label2id = {category:label for category, label in sorted(df[['category', 'label']].drop_duplicates().values, key = lambda x:x[1])}

NUM_LABELS = len(df['label'].drop_duplicates())
NUM_LABELS

77

In [4]:
data_texts = df['text'].tolist()
data_labels = df['label'].tolist()
df.head()

,text,category,label
0,I am still waiting on my card?,card_arrival,12
1,What can I do if my card still hasn't arrived ...,card_arrival,12
2,I have been waiting over a week. Is the card s...,card_arrival,12
3,Can I track my card while it is in the process...,card_arrival,12
4,"How do I know if I will get my card, or if it ...",card_arrival,12


In [5]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.2, random_state = 0 )

train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0 )


In [7]:
from transformers import DistilBertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Initialize the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=NUM_LABELS)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=7,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=100,
)

# Tokenize and encode the training and validation data (assuming you already have train_texts, val_texts)
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt'
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt'
)

# Convert labels to PyTorch tensors
train_labels_tensor = torch.tensor(train_labels)
val_labels_tensor = torch.tensor(val_labels)

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels_tensor)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels_tensor)

# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=12)

# Create a Trainer instance for PyTorch
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataloader=train_loader,  # Use train_loader directly
    eval_dataloader=val_loader,      # Use val_loader directly
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: ignored

In [6]:
from transformers import DistilBertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Initialize the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=NUM_LABELS)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=7,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=100,
)

# Tokenize and encode the training and validation data (assuming you already have train_texts, val_texts)
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt'
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt'
)

# Convert labels to PyTorch tensors
train_labels_tensor = torch.tensor(train_labels)
val_labels_tensor = torch.tensor(val_labels)

# Create PyTorch datasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels_tensor)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels_tensor)

# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=12)

# Create a Trainer instance for PyTorch
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_loader,
    eval_dataset=val_loader,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: ignored

In [ ]:
save_directory = "/saved_models"

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

In [ ]:
# Make predictions
test_texts = ["Your test text goes here"]
test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt'
)
predictions = trainer.predict(test_dataset=test_encodings)

# Extract confidence scores for each class
confidence_scores = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1)
confidence_scores = confidence_scores.detach().numpy()

# Get the class with the highest confidence score for each prediction
predicted_classes = np.argmax(confidence_scores, axis=1)


In [ ]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)
test_text = test_texts[0]

test_text

In [ ]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

In [ ]:
import torch

from transformers import DistilBertForSequenceClassification

tokenizer_fine_tuned_pt = DistilBertTokenizer.from_pretrained(save_directory)


model_fine_tuned_pt = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf = True )
predict_input_pt = tokenizer_fine_tuned_pt(test_text, truncation = True, padding = True, return_tensors = 'pt' )

ouput_pt = model_fine_tuned_pt(predict_input_pt)

prediction_value_pt = torch.argmax(ouput_pt[0], dim = 1 ).item()

prediction_value_pt